# 교차 검증
테스트 세트로 일반화 성능을 올바르게 예측하려면 다양한 테스트 세트가 필요함  
1. Data = Train + Validation + Test 
2. Train Set로 모델.fit()
3. Validation Set로 모델 평가()  
.  
.  
2-3 반복 -> 최적의 parmeter 선정  
.  
.  
4. Train + Validation Set로 최종 모델 피팅
5. Test Set로 최종 모델 평가 


## 검증세트(validation set)  
보통 train:validation:test = 20:20:60 으로 설정(유동적임!)

In [1]:
# 데이터 불러오기 
import pandas as pd
wine=pd.read_csv('http://bit.ly/wine_csv_data')

print(wine.shape)
wine.head()

(6497, 4)


,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
# 결측치 확인
wine.isnull().sum()

alcohol    0
sugar      0
pH         0
class      0
dtype: int64

In [3]:
# feature(X) - target(y) 나눠주기 
X=wine[['alcohol','sugar','pH']]
y=wine['class']

In [4]:
# train-test data 나눠주기
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5197, 3), (1300, 3), (5197,), (1300,))

In [6]:
# train-valid data 나눠주기 
X_sub,X_valid,y_sub,y_valid=train_test_split(X_train,y_train,test_size=0.2,random_state=42)
X_sub.shape, X_valid.shape, y_sub.shape, y_test.shape

((4157, 3), (1040, 3), (4157,), (1300,))

원래 훈련세트 : 5197개의 sample  
-> 훈련세트 : 4157개  
-> 검증세트 : 1040개 로 나눠짐

In [7]:
# 모델 훈련(with X_train,y_train)
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(X_sub,y_sub)

print(f'훈련세트 점수 : {dt.score(X_sub,y_sub)}')
print(f'검증세트 점수: {dt.score(X_valid,y_valid)}')

훈련세트 점수 : 0.9971133028626413
검증세트 점수: 0.8615384615384616


Overfitting!  
훈련세트 점수가 검증세트보다 많이 높음

## K-폴드 교차검증(K-Fold Cross Validation)  
+ 보통 5-fold CV, 10-fold CV 실시함  
+ k값이 커지면 검증세트 사이즈가 줄지만 괜차늠  

### cross_validate  
+ 훈련 데이터를 섞어서 폴드를 나누지 않음 
+ `train_test_split` 과정에서 데이터를 섞은 후 split 해서 괜찮지만  
+ CV과정에서 데이터를 섞으려면 `splitter`(분할기)를 지정해줘야 함

In [8]:
# 5-fodl CV
from sklearn.model_selection import cross_validate 
scores=cross_validate(dt,X_train,y_train) # cv=5 default
print(scores)

{'fit_time': array([0.03831291, 0.03667355, 0.02458453, 0.03871965, 0.01168704]), 'score_time': array([0.00486016, 0.00659657, 0.00595498, 0.01030517, 0.00327206]), 'test_score': array([0.86730769, 0.84903846, 0.87584216, 0.85563041, 0.83253128])}


결과 : 딕셔너리 반환  
+ `fit_time` : 모델 훈련 시간 
+ `socre_time` : 모델 검증 시간
+ `test_score` : 모델 평가 점수(검증세트 점수)-> 평균할거임
    + 이름은 test_score지만 검증점수임

In [9]:
# 검증 스코어 평균내기
import numpy as np 
print(np.mean(scores['test_score']))

0.8560700007403568


### `cross_validate()`의 splitter
+ Regression -> KFold splitter
+ Classifier -> StratifiedKFold splitter

In [10]:
from sklearn.model_selection import StratifiedKFold
splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
    # n_splits= k폴드 결정
scores=cross_validate(dt,X_train,y_train,cv=splitter)
np.mean(scores['test_score'])

0.8568434118867646

# 하이퍼파라미터 튜닝  
1. 라이브러리 기본값으로 모델 훈련&평가 
2. CV 점수를 통해 매개변수 조정  
    + 모델마다 1~2, 5~6개의 매개변수 존재 
    + 주의할 점!   
        + ex) decision tree의 `max_depth`의 최적값은 `min_samples_split`에 따라 달라진다 
        + 매개변수끼리 의존적임 
        + solution) `GridSearch`  
        

## GridSearchCV  
하이퍼파라미터 탐색 & 교차검증 모두 수행함

In [11]:
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]} 
    # min_impurity_decrease : decision tree 매개변수 
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

+ `cv`=5 기본값  
+ params 5개 지정했으므로 5x5=25개의 모델 훈련
+ `n_jobs`=-1 : 시스템 내 모든 코어 사용

In [12]:
gs.fit(X_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

+ `best_estimator_` : 검증 점수가 가장 높은 모델 저장되어 있음
+ `best_params_` : 최적의 매개변수 저장 
+ `cv_results_` : 각 파라미터에서 수행한 CV의 결과를 딕셔너리로 저장
    + 'mean_test_score' 키에는 5개(설정한 params 개수)의 검증 점수 평균이 저장


In [13]:
dt=gs.best_estimator_
dt.score(X_train,y_train)

0.9615162593804117

In [14]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [15]:
print(gs.cv_results_)
print(gs.cv_results_['mean_test_score'])

{'mean_fit_time': array([0.01188617, 0.02394309, 0.05231318, 0.0440227 , 0.03640718]), 'std_fit_time': array([0.00036723, 0.01886332, 0.01147412, 0.01879521, 0.01407763]), 'mean_score_time': array([0.00356398, 0.00510054, 0.01433601, 0.01294322, 0.02053814]), 'std_score_time': array([0.00015973, 0.0017821 , 0.00430495, 0.00800624, 0.01229325]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161694, 0.8825794 ]), 

In [16]:
# CV 점수 분석
sorted(gs.cv_results_['mean_test_score'],reverse=True)

[0.8681929740134745,
 0.867808913896498,
 0.8676160509365515,
 0.8649222625305398,
 0.8645361664322204]

-> 1st fold의 검증 점수가 제일 높음 

In [17]:
# 넘파이 argmax : 가장 큰 값의 인덱스 추출
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(best_index)

0


In [18]:
gs.cv_results_

{'mean_fit_time': array([0.01188617, 0.02394309, 0.05231318, 0.0440227 , 0.03640718]),
 'std_fit_time': array([0.00036723, 0.01886332, 0.01147412, 0.01879521, 0.01407763]),
 'mean_score_time': array([0.00356398, 0.00510054, 0.01433601, 0.01294322, 0.02053814]),
 'std_score_time': array([0.00015973, 0.0017821 , 0.00430495, 0.00800624, 0.01229325]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161

In [19]:
print(gs.cv_results_['params'][best_index])
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}
{'min_impurity_decrease': 0.0001}


-> 결과가 같음을 알 수 있음  
-> 검증 점수 평균이 가장 높은 모델의 param

### 중간 정리
__Grid Search Cross Validation__  
+ Cross Validation(교차검증) : 훈련세트를 k등분 하여 각각 다른 훈련세트 조합으로 모델을 학습시킴  
    -> k개의 모델을 k개의 검증세트로 점수를 매김  
    -> k개의 검증 점수를 평균  

+ Grid Search : 최적의 하이퍼파라미터를 찾기 위한 알고리즘  
    -> if, 4개의 하이퍼 파라미터 중에서 고민중이라면  
    -> 4개의 'mean_test_score'가 생성됨(by. `GridSearchCV`)  
    -> 그중 가장 높은 점수를 가진 모델이 최적 모델임  

__GridSearchCV 과정__  
1. 탐색할 hyperparameter 지정 
2. Train Set에서 Grid Search 수행 -> 최상의 평균 점수가 나오는 파라미터의 조합 찾기 
3. 최상의 매개변수 모델을 전체 훈련 세트를 사용해 최종 모델 훈련  
    -> 최종 모델도 GridSearchCV에 저장됨  


## 더욱 복잡한 GridSearch  
DecisionTreeClassifier의 매개변수 정보  
+ `min_impurity_decrease` : 노드 분할을 위한 불순도 감소 최소량 지정  
+ `max_depth` : 트리 깊이 제한 
+ `min_samples_split` : 노드를 나누기 위한 최소 샘플수

In [20]:
params={'min_impurity_decrease':np.arange(0.0001,0.001,0.0001), # start, stop, step
        'max_depth':range(5,20,1), # np.arange()는 소수도 가능
        'min_samples_split':range(2,100,10)}

In [21]:
np.arange(0.0001,0.001,0.0001)

array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009])

In [22]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(X_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
# GridSearch 결과
print(f'조합 개수 :', len(gs.cv_results_['mean_test_score']))

print('Top 10 검증 점수')
print(sorted(gs.cv_results_['mean_test_score'],reverse=True)[:10])


best_indx=np.argmax(gs.cv_results_['mean_test_score'])
print(f'{best_indx}번째 모델이 최적의 모델입니다.')
model=gs.best_estimator_
print(f'최적 모델의 검증 점수1 : {gs.best_score_}')
print(f'최적 모델의 검증 점수2 :', gs.cv_results_['mean_test_score'][best_indx])

print('최적의 param조합 :',end=' ')
print(gs.best_params_)

print(f'최종 모델의 훈련 점수 : {model.score(X_train,y_train)}')

조합 개수 : 1350
Top 10 검증 점수
[0.8683865773302731, 0.8683865773302731, 0.8683865773302731, 0.8683865773302731, 0.8683865773302731, 0.8683865773302731, 0.8681935292811135, 0.8681935292811135, 0.8681935292811135, 0.8681935292811135]
841번째 모델이 최적의 모델입니다.
최적 모델의 검증 점수1 : 0.8683865773302731
최적 모델의 검증 점수2 : 0.8683865773302731
최적의 param조합 : {'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
최종 모델의 훈련 점수 : 0.892053107562055


+ 한계 : 탐색할 parameter의 범위를 정해야함  
    + ex)
```
params={'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
        'max_depth':range(5,20,1),
        'min_samples_split':range(2,100,10)}
```
-> `랜덤서치`를 해보자!


# Random Search  
파라미터 값의 범위나 간격을 미리 정하기 어려울 때 유용함  
+ GridSearch : 파라미터 값 목록 전달
+ RandomSearch : 파라미터 샘플링할 수 있는 `확률분포` 객체 전달

## RS를 위한 uniform 분포 생성기

In [24]:
from scipy.stats import uniform,randint

+ uniform : 0~1 사이 실수 값 샘플링 ($f_X(x)=1$)
+ randint : 정수값 샘플링

In [25]:
rgen=randint(0,10) # 범위 지정
rgen.rvs(5) # 샘플링 개수 지정

array([8, 9, 4, 5, 1])

In [26]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 114, 103, 116, 100,  87, 106, 104,  91,  81]))

In [27]:
ugen=uniform(0,1)
ugen.rvs(10)

array([0.61022578, 0.28972573, 0.66539586, 0.32890803, 0.01936123,
       0.15401797, 0.93374789, 0.57563664, 0.87206593, 0.61969057])

## radom parameter dictionary

In [28]:
params={'min_impurity_decrease':uniform(0.0001,0.001),
        'max_depth':randint(20,50),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25)
        }

In [29]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                      params,n_iter=100, # 100개의 랜덤 파라미터로 서치할거임
                      n_jobs=-1,random_state=42)
gs.fit(X_train,y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5abf086d90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5abf0f2750>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5abf086210>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5abf086550>},
                   random_state=42)

In [30]:
# 최적 모델 결과
print('최적 파라미터')
print(gs.best_params_)
print(f'최적 모델 검증 점수 : {gs.best_score_}')

최적 파라미터
{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
최적 모델 검증 점수 : 0.8695428296438884


In [31]:
model=gs.best_estimator_ # 최적 모델
print(f'최종 모델 훈련 점수 : {model.score(X_train,y_train)}')
print(f'최종 모델 테스트 점수:{model.score(X_test,y_test)}')

최종 모델 훈련 점수 : 0.8928227823744468
최종 모델 테스트 점수:0.86


# 확인문제 3  


In [32]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
rs=RandomizedSearchCV(DecisionTreeClassifier(splitter='random'),
                      params,n_iter=100,
                      n_jobs=-1,random_state=42)

rs.fit(X_train,y_train)
model=rs.best_estimator_
print('최적 파라미터')
print(rs.best_params_
      
      
      
      
      )
print(f'최종 모델 훈련 점수 : {model.score(X_train,y_train)}')
print(f'최종 모델 테스트 점수:{model.score(X_test,y_test)}')

최적 파라미터
{'max_depth': 31, 'min_impurity_decrease': 0.0003517822958253642, 'min_samples_leaf': 2, 'min_samples_split': 4}
최종 모델 훈련 점수 : 0.8143159515104869
최종 모델 테스트 점수:0.7861538461538462
